In [1]:
import pandas as pd

## 1 
- Примените простое экспоненциальное сглаживание (SES) к курсу доллара (USD) для страны Australia с коэффициентом α = 0.2. 
- Выведите первое значение сглаженного ряда, округленное до двух знаков после запятой.

In [2]:
df = pd.read_csv('../data/06-1-daily.csv')

## 2
- Примените метод Хольта с аддитивным трендом к данным по авиаперевозкам.
- Выведите первое значение сглаженного ряда, округленное до двух знаков после запятой.

In [4]:
df = pd.read_csv('../data/06-2-airtravel.csv')